In [40]:
url = 'https://raw.githubusercontent.com/erisonsuzuki/llamaindex_pandas_query/refs/heads/main/content/vendas.csv'

In [41]:
import pandas as pd

In [42]:
dataframe = pd.read_csv(url)


In [43]:
dataframe.head() # Check the first 5 values

,ID_compra,filial,cidade,tipo_cliente,genero,tipo_produto,preco_unitario,quantidade,imposto_5%,total,data,hora,forma_pagamento,avaliacao
0,750-67-8428,A,Santo André,Membro,Feminino,Saúde e Beleza,74.69,7,26.1415,548.9715,2024-01-05,13:08:00,Carteira Digital,9.1
1,226-31-3081,C,São Caetano,Normal,Feminino,Eletrônicos,15.28,5,3.8200,80.2200,2024-03-08,10:29:00,Dinheiro,9.6
2,631-41-3108,A,Santo André,Normal,Masculino,Casa,46.33,7,16.2155,340.5255,2024-03-03,13:23:00,Cartão de Crédito,7.4
3,123-19-1176,A,Santo André,Membro,Masculino,Saúde e Beleza,58.22,8,23.2880,489.0480,2024-01-27,20:33:00,Carteira Digital,8.4
4,373-73-7910,A,Santo André,Normal,Masculino,Esportes e Viagem,86.31,7,30.2085,634.3785,2024-02-08,10:37:00,Carteira Digital,5.3


In [44]:
dataframe.info() # Check if there is null value

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID_compra        1000 non-null   object 
 1   filial           1000 non-null   object 
 2   cidade           1000 non-null   object 
 3   tipo_cliente     1000 non-null   object 
 4   genero           1000 non-null   object 
 5   tipo_produto     1000 non-null   object 
 6   preco_unitario   1000 non-null   float64
 7   quantidade       1000 non-null   int64  
 8   imposto_5%       1000 non-null   float64
 9   total            1000 non-null   float64
 10  data             1000 non-null   object 
 11  hora             1000 non-null   object 
 12  forma_pagamento  1000 non-null   object 
 13  avaliacao        1000 non-null   float64
dtypes: float64(4), int64(1), object(9)
memory usage: 109.5+ KB


In [45]:
dataframe.duplicated().sum() # Check if there is duplicated values

np.int64(0)

**LLamaindex**

Pandas query usage

Docs: https://developers.llamaindex.ai/python/examples/query_engine/pandas_query_engine

In [46]:
!pip -q install llama-index llama-index-experimental

Create groq key acessing this [link](https://console.groq.com/keys)

Supported models listed [here](https://console.groq.com/docs/models)

In [47]:
from google.colab import userdata
key = userdata.get('GROQ_API')

In [48]:
!pip -q install llama-index-llms-groq

In [49]:
from llama_index.core import Settings
from llama_index.llms.groq import Groq

Settings.llm = Groq(model='llama-3.3-70b-versatile', api_key=key)

In [50]:
from llama_index.experimental.query_engine import PandasQueryEngine

query_engine = PandasQueryEngine(df=dataframe, verbose=True)

In [52]:
response = query_engine.query(
    "Qual é a forma de pagamento mais utilizada?",
)

> Pandas Instructions:
```
df['forma_pagamento'].mode()[0]
```
> Pandas Output: Carteira Digital


In [58]:
dataframe['forma_pagamento'].value_counts() # Verifying the values in csv

,count
forma_pagamento,
Carteira Digital,345
Dinheiro,344
Cartão de Crédito,311


In [59]:
response = query_engine.query(
    "Qual o tipo de produto com maior quantidade por filial?",
)

> Pandas Instructions:
```
df.groupby(['filial', 'tipo_produto'])['quantidade'].sum().groupby('filial').idxmax()
```
> Pandas Output: filial
A                   (A, Casa)
B      (B, Esportes e Viagem)
C    (C, Alimentos e Bebidas)
Name: quantidade, dtype: object


In [64]:
dataframe.groupby(['filial', 'tipo_produto'])['quantidade'].sum() # Verifying the values in csv

filial  tipo_produto       
A       Alimentos e Bebidas    313
        Casa                   371
        Eletrônicos            322
        Esportes e Viagem      333
        Moda                   263
        Saúde e Beleza         257
B       Alimentos e Bebidas    270
        Casa                   295
        Eletrônicos            316
        Esportes e Viagem      322
        Moda                   297
        Saúde e Beleza         320
C       Alimentos e Bebidas    369
        Casa                   245
        Eletrônicos            333
        Esportes e Viagem      265
        Moda                   342
        Saúde e Beleza         277
Name: quantidade, dtype: int64

In [70]:
response = query_engine.query(
    "Qual a forma de pagamento mais utilizada em cada filial?",
)

> Pandas Instructions:
```
df.groupby('filial')['forma_pagamento'].apply(lambda x: x.mode().iloc[0])
```
> Pandas Output: filial
A    Carteira Digital
B    Carteira Digital
C            Dinheiro
Name: forma_pagamento, dtype: object


In [75]:
questions = [
    "Qual a forma de pagamento mais utilizada em cada filial?",
    "Em quais cidades temos filiais?",
    "Qual é o preço unitário médio de cada tipo de produto?",
    "Qual é a média de valor de compra por tipo de cliente?",
    "Qual é a filial com maior faturamento?"
]

for question in questions:
    print(f"\nPergunta: {question}")
    print(f"Resposta: {query_engine.query(question)}")


Pergunta: Qual a forma de pagamento mais utilizada em cada filial?
> Pandas Instructions:
```
df.groupby('filial')['forma_pagamento'].apply(lambda x: x.mode().iloc[0])
```
> Pandas Output: filial
A    Carteira Digital
B    Carteira Digital
C            Dinheiro
Name: forma_pagamento, dtype: object
Resposta: filial
A    Carteira Digital
B    Carteira Digital
C            Dinheiro
Name: forma_pagamento, dtype: object

Pergunta: Em quais cidades temos filiais?
> Pandas Instructions:
```
df['cidade'].unique()
```
> Pandas Output: ['Santo André' 'São Caetano' 'São Bernardo do Campo']
Resposta: ['Santo André' 'São Caetano' 'São Bernardo do Campo']

Pergunta: Qual é o preço unitário médio de cada tipo de produto?
> Pandas Instructions:
```
df.groupby('tipo_produto')['preco_unitario'].mean()
```
> Pandas Output: tipo_produto
Alimentos e Bebidas    56.008851
Casa                   55.316937
Eletrônicos            53.551588
Esportes e Viagem      56.993253
Moda                   57.153652
Saúde